In [1]:
using FFTW, Plots, NAJ

In [2]:
function conv1d(u::AbstractVector{T}, v::AbstractVector{S}) where {T<:Number, S<:Number}
    M, N = length(u), length(v)
    TT = promote_type(T, S)
    r = zeros(TT, M+N)

    for i ∈ 1:M, j = 1:N
        r[i+j] += u[i]*v[j]
    end
    return r
end

function xcorr1d(u::AbstractVector{T}, v::AbstractVector{S}) where {T<:Number, S<:Number}
    return conv1d(u, v[end:-1:1])
end

xcorr1d (generic function with 1 method)

In [18]:
function _dft(
    f::Vector{T}, 
    inverse = false) where T<:Union{Real, Complex}
    
    N = length(f)
    if T <:AbstractFloat
        F = zeros(Complex{T}, N)
    elseif T<:Complex 
        F = zeros(eltype(f), N)
    else 
        F = zeros(Complex{Float64}, N)
    end

    if inverse
        for i in 1:N
            F[i] = [f[k] * exp(2.0im * π* (i-1) * (k-1)/N) for k ∈ 1:N]./N |> sum
        end
    else     
        for i in 1:N
            F[i] = [f[k] * exp(-2.0im * π* (i-1) * (k-1)/N) for k ∈ 1:N] |> sum
        end
    end

    return F
end

dft = v-> _dft(v, false)
idft = v-> _dft(v, true)

#33 (generic function with 1 method)

In [20]:
f=[0, 0, 0, 1, 0, 0, 0, 0]
g = [1,2, 4, 2, 8]
conv1d(f, g)

13-element Vector{Int64}:
 0
 0
 0
 0
 1
 2
 4
 2
 8
 0
 0
 0
 0

In [26]:
fft(f) .- dft(f)

8-element Vector{ComplexF64}:
                     0.0 + 0.0im
 -1.1102230246251565e-16 + 0.0im
  1.8369701987210297e-16 + 0.0im
 -1.1102230246251565e-16 - 2.220446049250313e-16im
                     0.0 + 3.6739403974420594e-16im
   9.992007221626409e-16 - 8.881784197001252e-16im
   -5.51091059616309e-16 + 0.0im
  -2.220446049250313e-16 - 1.1102230246251565e-16im

In [22]:
idft(f)

8-element Vector{ComplexF64}:
                  0.125 + 0.0im
   -0.08838834764831843 + 0.08838834764831845im
 -2.296212748401287e-17 - 0.125im
    0.08838834764831846 + 0.08838834764831842im
                 -0.125 + 4.592425496802574e-17im
    0.08838834764831832 - 0.08838834764831856im
  6.888638245203862e-17 + 0.125im
   -0.08838834764831842 - 0.08838834764831846im

In [23]:
h = rand(10)

10-element Vector{Float64}:
 0.9351749060533987
 0.002047843477063216
 0.6580701413359334
 0.37879246391960775
 0.4679394731725509
 0.7605640918375494
 0.9411106014088723
 0.8519443739462411
 0.06801298650937204
 0.4716144523244771

In [24]:
ifft(fft(h))

10-element Vector{ComplexF64}:
    0.9351749060533987 + 0.0im
 0.0020478434770632606 + 0.0im
    0.6580701413359334 + 0.0im
   0.37879246391960786 + 0.0im
     0.467939473172551 + 0.0im
    0.7605640918375495 + 0.0im
    0.9411106014088724 + 0.0im
    0.8519443739462415 + 0.0im
   0.06801298650937197 + 0.0im
   0.47161445232447713 + 0.0im

In [12]:
idft(dft(h))

10-element Vector{ComplexF64}:
  0.1882140158569331 + 7.632783294297951e-17im
 0.05313304298272799 + 1.3877787807814457e-16im
   0.689754975953263 - 1.942890293094024e-16im
  0.8175720934836052 + 0.0im
  0.7322895313119051 + 1.8041124150158794e-16im
  0.8742454671318001 - 3.400058012914542e-16im
  0.7844929594746285 - 2.498001805406602e-16im
  0.5400707965507728 + 3.608224830031759e-16im
 0.28024567250734467 + 1.1102230246251565e-16im
 0.24357565170020765 + 8.881784197001252e-16im